<a href="https://colab.research.google.com/github/Nasik80/brototype_competition_entry/blob/main/Project_Installer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# --- CONFIGURATION ---
PROJECT_NAME = "brototype_competition_entry"

# --- FILE CONTENTS ---

# 1. DJANGO MODELS
models_py = """from django.db import models

class Complaint(models.Model):
    PRIORITY_CHOICES = [('LOW', 'Low'), ('MED', 'Medium'), ('HIGH', 'High')]
    STATUS_CHOICES = [('OPEN', 'Open'), ('RESOLVED', 'Resolved')]

    student_id = models.CharField(max_length=20)
    batch = models.CharField(max_length=50)
    domain = models.CharField(max_length=50)
    category = models.CharField(max_length=50)
    description = models.TextField()
    priority = models.CharField(max_length=4, choices=PRIORITY_CHOICES, default='MED')
    status = models.CharField(max_length=10, choices=STATUS_CHOICES, default='OPEN')
    created_at = models.DateTimeField(auto_now_add=True)

    def __str__(self):
        return f"{self.student_id} - {self.category}"
"""

# 2. DJANGO SERIALIZERS
serializers_py = """from rest_framework import serializers
from .models import Complaint

class ComplaintSerializer(serializers.ModelSerializer):
    class Meta:
        model = Complaint
        fields = '__all__'
"""

# 3. DJANGO VIEWS
views_py = """from rest_framework import viewsets
from rest_framework.permissions import AllowAny, IsAuthenticated
from .models import Complaint
from .serializers import ComplaintSerializer

class ComplaintViewSet(viewsets.ModelViewSet):
    queryset = Complaint.objects.all().order_by('-created_at')
    serializer_class = ComplaintSerializer

    def get_permissions(self):
        # Allow anyone to POST (submit), only Admin can GET/PUT/DELETE
        if self.action == 'create':
            return [AllowAny()]
        return [AllowAny()] # For competition demo purposes, allowing all. Change to IsAuthenticated() for production.
"""

# 4. DJANGO URLS
urls_py = """from django.urls import path, include
from rest_framework.routers import DefaultRouter
from .views import ComplaintViewSet

router = DefaultRouter()
router.register(r'complaints', ComplaintViewSet)

urlpatterns = [
    path('', include(router.urls)),
]
"""

# 5. REACT FRONTEND (App.jsx)
# Note: This version is pre-wired for Django (port 8000)
react_app_jsx = """import React, { useState, useEffect, useRef } from 'react';
import * as THREE from 'three';
import { AlertCircle, CheckCircle, Terminal, Cpu, Shield, Code, ChevronRight, Loader2, Lock, Users, Activity, X, Check, LogOut } from 'lucide-react';

// --- API CONFIG ---
const API_URL = 'http://localhost:8000/api/complaints/';

const App = () => {
  const [view, setView] = useState('student');
  const [mounted, setMounted] = useState(false);

  useEffect(() => { setMounted(true); }, []);

  // --- API HANDLERS ---
  const submitComplaint = async (data) => {
    try {
      const response = await fetch(API_URL, {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify(data)
      });
      if (!response.ok) throw new Error('Network response was not ok');
      return true;
    } catch (error) {
      console.error("API Error (Ensure Django is running):", error);
      // Fallback for demo if backend is offline
      alert("Backend offline? Check console. simulating success.");
      return true;
    }
  };

  const fetchComplaints = async () => {
    try {
      const response = await fetch(API_URL);
      if (!response.ok) throw new Error('Failed to fetch');
      return await response.json();
    } catch (error) {
      console.error("API Error:", error);
      return [];
    }
  };

  // --- RENDERER ---
  const renderView = () => {
    switch(view) {
      case 'student': return <StudentPortal onNavigate={setView} onSubmit={submitComplaint} />;
      case 'login': return <AdminLogin onLogin={() => setView('admin')} onBack={() => setView('student')} />;
      case 'admin': return <AdminDashboard onLogout={() => setView('student')} fetcher={fetchComplaints} />;
      default: return <StudentPortal onNavigate={setView} />;
    }
  };

  return (
    <div className="relative min-h-screen w-full bg-black text-white overflow-hidden font-sans">
      <ThreeBackground view={view} />
      {/* Header */}
      <header className="fixed top-0 left-0 w-full p-6 flex justify-between items-center z-50 pointer-events-none">
        <div className="flex items-center gap-3 pointer-events-auto cursor-pointer" onClick={() => setView('student')}>
          <div className="w-10 h-10 bg-white text-black flex items-center justify-center font-bold text-2xl rounded-sm">B</div>
          <div className="flex flex-col"><span className="font-bold text-sm uppercase">Brototype</span><span className="text-[10px] text-gray-400">KOCHI // HQ</span></div>
        </div>
        <div className="flex items-center gap-4 text-xs font-mono text-gray-400 bg-black/50 backdrop-blur-md px-4 py-2 rounded-full border border-white/10">
          <span>SYS.STATUS: ONLINE</span>
          <span className={`w-2 h-2 rounded-full ${view === 'admin' ? 'bg-red-500' : 'bg-green-500'} animate-pulse`}></span>
        </div>
      </header>
      <div className={`relative z-10 transition-all duration-1000 ${mounted ? 'opacity-100 translate-y-0' : 'opacity-0 translate-y-10'}`}>{renderView()}</div>
    </div>
  );
};

// --- COMPONENTS ---
const StudentPortal = ({ onNavigate, onSubmit }) => {
  const [form, setForm] = useState({ student_id: '', batch: '', domain: 'python', category: 'infrastructure', description: '' });
  const [loading, setLoading] = useState(false);
  const [success, setSuccess] = useState(false);

  const handleSubmit = async (e) => {
    e.preventDefault();
    setLoading(true);
    await onSubmit(form);
    setLoading(false);
    setSuccess(true);
  };

  if (success) return (
    <div className="flex flex-col items-center justify-center min-h-screen">
      <CheckCircle size={64} className="text-white mb-4" />
      <h1 className="text-3xl font-bold">REPORT SUBMITTED</h1>
      <button onClick={() => setSuccess(false)} className="mt-4 text-sm border-b border-gray-500">Back</button>
    </div>
  );

  return (
    <div className="flex flex-col items-center justify-center min-h-screen p-4 pt-20">
       <h1 className="text-5xl md:text-7xl font-black tracking-tighter text-transparent bg-clip-text bg-gradient-to-b from-white to-gray-800 mb-8">GRIEVANCE<br/>PORTAL</h1>
       <div className="w-full max-w-xl bg-black/40 backdrop-blur-md border border-white/10 p-8 relative">
         <form onSubmit={handleSubmit} className="space-y-4">
           <div className="grid grid-cols-2 gap-4">
             <input required placeholder="Student ID" className="bg-white/5 p-3 border-b border-white/20 text-white outline-none" onChange={e => setForm({...form, student_id: e.target.value})} />
             <input required placeholder="Batch" className="bg-white/5 p-3 border-b border-white/20 text-white outline-none" onChange={e => setForm({...form, batch: e.target.value})} />
           </div>
           <select className="w-full bg-white/5 p-3 border-b border-white/20 text-white outline-none" onChange={e => setForm({...form, domain: e.target.value})}>
             <option value="python">Python Django</option><option value="mern">MERN Stack</option><option value="flutter">Flutter</option>
           </select>
           <select className="w-full bg-white/5 p-3 border-b border-white/20 text-white outline-none" onChange={e => setForm({...form, category: e.target.value})}>
             <option value="infrastructure">Infrastructure</option><option value="academic">Academic</option><option value="wifi">Network/Wifi</option>
           </select>
           <textarea required rows="4" placeholder="Description" className="w-full bg-white/5 p-3 border border-white/10 text-white outline-none" onChange={e => setForm({...form, description: e.target.value})}></textarea>
           <button disabled={loading} className="w-full bg-white text-black font-bold py-3 hover:bg-gray-200 flex justify-center">
             {loading ? <Loader2 className="animate-spin"/> : "SUBMIT REPORT"}
           </button>
         </form>
         <div className="mt-6 text-center"><button onClick={() => onNavigate('login')} className="text-[10px] text-gray-500 uppercase flex items-center justify-center gap-2 w-full"><Lock size={10}/> Admin Access</button></div>
       </div>
    </div>
  );
};

const AdminLogin = ({ onLogin, onBack }) => {
  const [pass, setPass] = useState('');
  const handle = (e) => { e.preventDefault(); if(pass==='admin') onLogin(); else alert('Wrong password'); };
  return (
    <div className="flex flex-col items-center justify-center min-h-screen bg-black/90 z-50">
      <div className="border border-red-500/30 p-8 text-center">
        <h2 className="text-red-500 text-xl font-bold mb-4 uppercase tracking-widest">Restricted Area</h2>
        <form onSubmit={handle} className="space-y-4">
          <input autoFocus type="password" placeholder="PASSCODE" className="bg-black border border-red-900 text-red-500 text-center p-2 w-full tracking-widest outline-none" value={pass} onChange={e=>setPass(e.target.value)}/>
          <button className="bg-red-900/20 text-red-500 border border-red-500 w-full py-2 font-bold hover:bg-red-500 hover:text-white">AUTHENTICATE</button>
        </form>
        <button onClick={onBack} className="mt-4 text-gray-500 text-xs">CANCEL</button>
      </div>
    </div>
  );
};

const AdminDashboard = ({ onLogout, fetcher }) => {
  const [data, setData] = useState([]);

  useEffect(() => {
    fetcher().then(setData);
  }, []);

  return (
    <div className="min-h-screen pt-24 px-8 flex flex-col items-center">
      <div className="w-full max-w-6xl">
        <div className="flex justify-between items-end border-b border-white/10 pb-4 mb-8">
          <h1 className="text-3xl font-bold">COMMAND CENTER</h1>
          <button onClick={onLogout} className="text-red-500 text-xs border border-red-500/30 px-4 py-2 hover:bg-red-500 hover:text-white">DISCONNECT</button>
        </div>
        <div className="grid grid-cols-1 md:grid-cols-3 gap-4 mb-8">
           <div className="bg-white/5 p-6 border border-white/10"><div className="text-gray-500 text-xs">TOTAL</div><div className="text-4xl font-bold">{data.length}</div></div>
           <div className="bg-white/5 p-6 border border-white/10"><div className="text-gray-500 text-xs">OPEN</div><div className="text-4xl font-bold text-orange-500">{data.filter(i=>i.status==='OPEN').length}</div></div>
           <div className="bg-white/5 p-6 border border-white/10"><div className="text-gray-500 text-xs">HIGH PRIORITY</div><div className="text-4xl font-bold text-red-500">{data.filter(i=>i.priority==='HIGH').length}</div></div>
        </div>
        <div className="bg-white/5 border border-white/10">
           {data.map(item => (
             <div key={item.id} className="grid grid-cols-12 gap-4 p-4 border-b border-white/5 hover:bg-white/5 items-center">
               <div className="col-span-1 text-gray-500 text-xs">#{item.id}</div>
               <div className="col-span-2 font-bold">{item.student_id}</div>
               <div className="col-span-2 text-xs uppercase border border-white/20 text-center py-1">{item.domain}</div>
               <div className="col-span-5 text-gray-400 text-sm">{item.description}</div>
               <div className="col-span-2 text-right text-xs font-bold">{item.status}</div>
             </div>
           ))}
           {data.length === 0 && <div className="p-8 text-center text-gray-500">NO DATA / BACKEND OFFLINE</div>}
        </div>
      </div>
    </div>
  );
};

const ThreeBackground = ({view}) => {
  const m = useRef(null);
  useEffect(() => {
    const scene = new THREE.Scene();
    const cam = new THREE.PerspectiveCamera(75, window.innerWidth/window.innerHeight, 0.1, 1000);
    const ren = new THREE.WebGLRenderer({alpha:true});
    ren.setSize(window.innerWidth, window.innerHeight);
    m.current.appendChild(ren.domElement);
    const geo = new THREE.PlaneGeometry(100,100,40,40);
    const mat = new THREE.MeshBasicMaterial({color:0xffffff, wireframe:true, transparent:true, opacity:0.1});
    const plane = new THREE.Mesh(geo,mat);
    plane.rotation.x = -Math.PI/2; plane.position.y = -10; scene.add(plane);
    cam.position.set(0,5,20);
    const anim = () => { requestAnimationFrame(anim); plane.rotation.z += 0.001; ren.render(scene,cam); };
    anim();
    return () => { if(m.current) m.current.innerHTML = ''; };
  }, []);
  return <div ref={m} className="fixed top-0 left-0 w-full h-full -z-10"/>;
};

export default App;
"""

# --- INSTALLER LOGIC ---

def create_file(path, content):
    with open(path, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"Created: {path}")

def main():
    # 1. Create Directories
    dirs = [
        f"{PROJECT_NAME}/backend/complaints",
        f"{PROJECT_NAME}/frontend/src",
    ]

    for d in dirs:
        os.makedirs(d, exist_ok=True)

    # 2. Create Django Files
    create_file(f"{PROJECT_NAME}/backend/complaints/models.py", models_py)
    create_file(f"{PROJECT_NAME}/backend/complaints/serializers.py", serializers_py)
    create_file(f"{PROJECT_NAME}/backend/complaints/views.py", views_py)
    create_file(f"{PROJECT_NAME}/backend/complaints/urls.py", urls_py)

    # Create empty __init__.py files
    create_file(f"{PROJECT_NAME}/backend/complaints/__init__.py", "")

    # 3. Create Frontend Files
    create_file(f"{PROJECT_NAME}/frontend/src/App.jsx", react_app_jsx)

    # 4. Create Instructions
    readme = """# Brototype Grievance Portal - Setup

## 1. Backend (Django)
1. Open terminal in `backend/`
2. Run: `django-admin startproject core .`
3. Add 'rest_framework', 'corsheaders', and 'complaints' to INSTALLED_APPS in `settings.py`.
4. Add `path('api/', include('complaints.urls'))` to `core/urls.py`.
5. Run: `python manage.py makemigrations`
6. Run: `python manage.py migrate`
7. Run: `python manage.py runserver`

## 2. Frontend (React)
1. Create a Vite app: `npm create vite@latest` (select React)
2. Replace `src/App.jsx` with the file provided in `frontend/src/App.jsx`.
3. Run: `npm install three lucide-react`
4. Run: `npm run dev`
    """
    create_file(f"{PROJECT_NAME}/README.txt", readme)

    print(f"\\nSUCCESS! Project created in folder: {PROJECT_NAME}")
    print("Check README.txt inside the folder for next steps.")

if __name__ == "__main__":
    main()

Created: brototype_competition_entry/backend/complaints/models.py
Created: brototype_competition_entry/backend/complaints/serializers.py
Created: brototype_competition_entry/backend/complaints/views.py
Created: brototype_competition_entry/backend/complaints/urls.py
Created: brototype_competition_entry/backend/complaints/__init__.py
Created: brototype_competition_entry/frontend/src/App.jsx
Created: brototype_competition_entry/README.txt
\nSUCCESS! Project created in folder: brototype_competition_entry
Check README.txt inside the folder for next steps.
